In [0]:
!nvidia-smi

Sun Apr 14 20:21:14 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    73W / 149W |   7449MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers

import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

print(tf.__version__) 
tf.enable_eager_execution

Mounted at /content/drive
1.13.1


<function tensorflow.python.framework.ops.enable_eager_execution>

In [0]:
train_x = np.load('/content/drive/My Drive/20564939_dsikka/train_x.npy')
train_labels = np.load('/content/drive/My Drive/20564939_dsikka/train_label.npy')
test_new_data = np.load('/content/drive/My Drive/20564939_dsikka/test_x.npy')
classes = np.unique(train_labels)


In [0]:
# Add callback with early stopping to prevent overfitting
callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience=4, mode='auto'),
             ModelCheckpoint(filepath='/content/drive/My Drive/20564939_dsikka/model.h5', monitor='val_loss', save_best_only=True)]

X_train, X_test, y_train, y_test = train_test_split(train_x, train_labels, test_size=0.20,random_state=np.random.RandomState(), stratify=train_labels)

In [0]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(168, 308, 3)),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.BatchNormalization(),
 
    keras.layers.Conv2D(96, (3, 3), activation='relu'),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Flatten(),
    
    #Regularizer (L2) to prevent overfitting in fully connected layers
    keras.layers.Dense(120, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(120, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(len(classes), activation='softmax', kernel_regularizer=regularizers.l2(0.01))
])

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [0]:
model.fit(X_train, y_train, callbacks=callbacks, epochs=20, validation_data=(X_test,y_test), verbose=1)

Train on 608 samples, validate on 152 samples
Epoch 1/20
608/608 [==============================] - 23s 38ms/sample - loss: 5.9556 - acc: 0.3322 - val_loss: 17.0109 - val_acc: 0.0526
Epoch 2/20
608/608 [==============================] - 3s 5ms/sample - loss: 4.7000 - acc: 0.6711 - val_loss: 11.4099 - val_acc: 0.0724
Epoch 3/20
608/608 [==============================] - 3s 5ms/sample - loss: 4.1725 - acc: 0.7845 - val_loss: 8.8281 - val_acc: 0.0855
Epoch 4/20
608/608 [==============================] - 3s 5ms/sample - loss: 3.7795 - acc: 0.8421 - val_loss: 7.2104 - val_acc: 0.1908
Epoch 5/20
608/608 [==============================] - 3s 5ms/sample - loss: 3.4955 - acc: 0.8882 - val_loss: 7.3391 - val_acc: 0.1711
Epoch 6/20
608/608 [==============================] - 3s 5ms/sample - loss: 3.1961 - acc: 0.9062 - val_loss: 6.2999 - val_acc: 0.2829
Epoch 7/20
608/608 [==============================] - 3s 5ms/sample - loss: 3.0236 - acc: 0.9145 - val_loss: 5.4460 - val_acc: 0.3026
Epoch 8/20
6

In [0]:
# READ ME

# The following code loads the model with the best validation score that was JUST
# trained above. This is not necessarily the best model or the model used for 
# Kaggle. This is simply the model which contains the weights for this particular
# training run. The model with the best score is shown below.
# The results for this model for testing data given are stored in 'current_result.csv' 
# in the 20564939_dsikka directory

import pandas as pd
model_saved = keras.models.load_model('/content/drive/My Drive/20564939_dsikka/model.h5')
current_result = model_saved.predict(X_test)
a = np.argmax(current_result, axis=1)

Id = np.array(list(range(len(a))))

df = pd.DataFrame([Id, a]).T
df.columns = ['Id', 'Predicted']
df.to_csv('/content/drive/My Drive/20564939_dsikka/current_result.csv', index = False)

In [0]:
# READ ME

# The following code loads a pretrained model which I trained and found that it has the 
# which has the weights that allowed the
# BEST Kaggle score for the public leaderboard. For the purpose of testing
# on the private leaderboard, this model should be used, not the above.
# The results for this model for testing data given are stored in 'result.csv' 
# in the 20564939_dsikka directory and it is csv used for kaggle


import pandas as pd
best_model = keras.models.load_model('/content/drive/My Drive/20564939_dsikka/best_model_93.h5')
result = best_model.predict(test_new_data)
a = np.argmax(result, axis=1)

Id = np.array(list(range(len(a))))

df = pd.DataFrame([Id, a]).T
df.columns = ['Id', 'Predicted']
df.to_csv('/content/drive/My Drive/20564939_dsikka/result.csv', index = False)